# Programming with `geoh5` (ANALYST)


This section provides examples on how to interact with Geoscience ANALYST programmatically.

Participants will learn how to read and write to a geoh5 using [geoh5py](https://geoh5py.readthedocs.io/en/stable/)

- Managing a Workspace
- Entities
- Example

 Copyright (c) 2022 Mira Geoscience Ltd.